In [2]:
import os
from Bio import SeqIO

In [3]:
def ucitaj_genome(putanja_do_foldera):
    genomi_kodovi = {}
    for ime_fajla in os.listdir(putanja_do_foldera): 
        if ime_fajla.endswith(".fasta"):
            putanja_do_fajla = os.path.join(putanja_do_foldera, ime_fajla)
            with open(putanja_do_fajla, "r") as fajl:
                virus = ime_fajla[:-6]
                for genom in SeqIO.parse(fajl, "fasta"):
                    genomi_kodovi[virus] = genom.seq

    return genomi_kodovi

In [4]:
def ucitaj_proteine(putanja_do_foldera):
    proteini_kodovi = {}
    for ime_fajla in os.listdir(putanja_do_foldera): 
        if ime_fajla.endswith(".fasta"):
            putanja_do_fajla = os.path.join(putanja_do_foldera, ime_fajla)
            with open(putanja_do_fajla, "r") as fajl:
                virus = ime_fajla[:-6]
                for protein in SeqIO.parse(fajl, "fasta"):
                    proteini_kodovi[virus] = protein.seq

    return proteini_kodovi

In [5]:
# Putanja do direktorijuma sa fasta fajlovima
putanja_do_foldera = "podaci/genomi/referentni_modeli"
# Učitavanje genoma iz datoteka
genomi_kodovi = ucitaj_genome(putanja_do_foldera)

# Putanja do direktorijuma sa fasta fajlovima
putanja_do_foldera = "podaci/proteini/referentni_modeli"
# Učitavanje proteina iz datoteka
proteini_kodovi = ucitaj_proteine(putanja_do_foldera)

In [7]:
for virus, kod in proteini_kodovi.items():
    print(virus, len(kod))

mers 1353
bcov 1363
sars1 1255
human229e 1173
humanoc43 1353


In [ ]:
for virus, kod in genomi_kodovi.items():
    print(virus, len(kod))

In [19]:
from Bio.Seq import Seq

# S obzirom da su sekvenca surface glycoproteina sars1, mers, human 229e, human oc43 i bcov virusa kodirane sa 
# 1255, 1353, 1173, 1353, 1363 aminokiselina redom, dovoljno je da trazimo otvorena citanja koja su duza od 1000 kodona
def pronadji_orf(sekvencu, min_duzina=1000):
    # Pronalaženje svih ORF (koji su duzi od min_duzina kodona) u DNK sekvenci
    start_kodon = 'ATG'
    stop_kodonovi = ['TAA', 'TAG', 'TGA']
    orfovi = []
    
    # Pronalaženje svih pozicija start kodona
    start_pozicije = [i for i in range(len(sekvencu) - 2) if sekvencu[i:i+3] == start_kodon]
    
    for start_poz in start_pozicije:
        orf = ''
        for i in range(start_poz, len(sekvencu) - 2, 3):
            kodon = sekvencu[i:i+3]
            if kodon in stop_kodonovi:
                if len(orf) >= min_duzina:
                    orfovi.append((start_poz, i+3, orf))
                break
            orf += kodon
    return orfovi


def prevedi_sekvencu(sekvenca):
    # Prevodi RNK sekvencu u sekvencu aminokiselina.
    tabela_kodona = {
        'AUU':'I', 'AUC':'I', 'AUA':'I', 'AUG':'M',
        'ACU':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T',
        'AAU':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K',
        'AGU':'S', 'AGC':'S', 'AGA':'R', 'AGG':'R',
        'GUU':'V', 'GUC':'V', 'GUA':'V', 'GUG':'V',
        'GCU':'A', 'GCC':'A', 'GCA':'A', 'GCG':'A',
        'GAU':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E',
        'GGU':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G',
        'UCU':'S', 'UCC':'S', 'UCA':'S', 'UCG':'S',
        'UUU':'F', 'UUC':'F', 'UUA':'L', 'UUG':'L',
        'UAU':'Y', 'UAC':'Y', 'UAA':'*', 'UAG':'*',
        'UGU':'C', 'UGC':'C', 'UGA':'*', 'UGG':'W',
        'CCU':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P',
        'CAU':'H', 'CAC':'H', 'CAA':'Q', 'CAG':'Q',
        'CGU':'R', 'CGC':'R', 'CGA':'R', 'CGG':'R',
        'CUU':'L', 'CUC':'L', 'CUA':'L', 'CUG':'L'
    }
    
    sekv_aminokiselina = ''
    for i in range(0, len(sekvenca), 3):
        kodon = sekvenca[i:i+3]
        aminokiselina = tabela_kodona.get(kodon) 
        sekv_aminokiselina += aminokiselina
    return sekv_aminokiselina

In [20]:
proteinske_sekvence_u_genomima = {}
for virus, genom_kod in genomi_kodovi.items():  
    # Pronalaženje ORF-ova u sekvenci
    orfovi = pronadji_orf(genom_kod)
    
    # Prevodjenje ORF-ova u aminokiseline i ispis rezultata
    for start, end, orf_sekv in orfovi:
        # Transkripcija DNK u RNK
        rnk_sekv_proteina = Seq(orf_sekv).transcribe()
        # Prevodjenje RNK u protein
        sekv_aminokiselina = prevedi_sekvencu(rnk_sekv_proteina)
        
        if sekv_aminokiselina == proteini_kodovi[virus]:
            # print(f"ORF od pozicije {start} do {end} preveden u sekvencu aminokiselina virusa {virus}")
            proteinske_sekvence_u_genomima[virus] = rnk_sekv_proteina

In [21]:
# Prevodjenje kodona u p-adicne brojeve
def kodon_u_p_adicni_broj(kodon):
    kodon_u_broj = {
        'CCC': 111, 'CCU': 113, 'CCA': 112, 'CCG': 114,
        'CAC': 121, 'CAU': 123, 'CAA': 122, 'CAG': 124,
        'CUC': 131, 'CUU': 133, 'CUA': 132, 'CUG': 134,
        'CGC': 141, 'CGU': 143, 'CGA': 142, 'CGG': 144,
        'ACC': 211, 'ACU': 213, 'ACA': 212, 'ACG': 214,
        'AAC': 221, 'AAU': 223, 'AAA': 222, 'AAG': 224,
        'AUC': 231, 'AUU': 233, 'AUA': 232, 'AUG': 234,
        'AGC': 241, 'AGU': 243, 'AGA': 242, 'AGG': 244,
        'UCC': 311, 'UCU': 313, 'UCA': 312, 'UCG': 314,
        'UAC': 321, 'UAU': 323, 'UAA': 322, 'UAG': 324,
        'UUC': 331, 'UUU': 333, 'UUA': 332, 'UUG': 334,
        'UGC': 341, 'UGU': 343, 'UGA': 342, 'UGG': 344,
        'GCC': 411, 'GCU': 413, 'GCA': 412, 'GCG': 414,
        'GAC': 421, 'GAU': 423, 'GAA': 422, 'GAG': 424,
        'GUC': 431, 'GUU': 433, 'GUA': 432, 'GUG': 434,
        'GGC': 441, 'GGU': 443, 'GGA': 442, 'GGG': 444
    }

    return kodon_u_broj[kodon]
        
# Prevodjenje RNK u niz p-adicnih brojeva
def rnk_sekvencu_u_p_adicne_brojeve(rnk_sekvenca):
    brojevi = []
    for i in range(0, len(rnk_sekvenca), 3):
        kodon = rnk_sekvenca[i:i+3]
        brojevi.append(kodon_u_p_adicni_broj(kodon))  
    return brojevi

In [22]:
def p_adicno_rastojanje_kodona(x, y, p):
    x, y = str(x), str(y)
    
    if x[0] != y[0]:
        # print(x, y, 1)
        return 1
    elif x[1] != y[1]:
        # print(x, y, 1/p)
        return 1/p
    elif x[2] != y[2]:
        # print(x, y, 1/p**2)
        return 1/(p**2)

    # print(x, y, 0)
    return 0
    
def p_adicno_rastojanje(protein1, protein2, p=5):
    p_rastojanje = 0
    for (x, y) in zip(protein1, protein2):
        p_rastojanje += p_adicno_rastojanje_kodona(x, y, p)

    return p_rastojanje

In [23]:
import numpy as np

humanoc43 = np.array(rnk_sekvencu_u_p_adicne_brojeve(proteinske_sekvence_u_genomima['humanoc43']))
mers = np.array(rnk_sekvencu_u_p_adicne_brojeve(proteinske_sekvence_u_genomima['mers']))

print(humanoc43.size)
print(p_adicno_rastojanje(humanoc43, mers))

1353
1061.7600000000048


In [24]:
def rnk_u_kodoni(rnk_sekvenca):
    kodoni = []
    for i in range(0, len(rnk_sekvenca), 3):
        kodon = rnk_sekvenca[i:i+3]
        kodoni.append(kodon)  
    return kodoni

In [25]:
humanoc43_codons = rnk_u_kodoni(proteinske_sekvence_u_genomima['humanoc43'])
mers_codons = rnk_u_kodoni(proteinske_sekvence_u_genomima['mers'])

In [26]:
hamming = 0
for x, y in zip(mers_codons, humanoc43_codons):
    if str(x) == str(y):
        # print(x, y, 0)
        hamming += 0
    else:
        # print(x, y, 1)
        hamming += 1

print(hamming)
    

1316
